# Fase 3 - Preparação dos Dados

## 0. Importando Bibliotecas Externas e o Dataset

In [22]:
import warnings
warnings.filterwarnings("ignore")

In [23]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.io import arff
from sklearn.model_selection import train_test_split


RANDOM = 51

### Importando os dados

In [24]:
file_path = "../data/raw/electricity-normalized.arff"
data, meta = arff.loadarff(file_path)

df = pd.DataFrame(data)

df

,date,day,period,nswprice,nswdemand,vicprice,vicdemand,transfer,class
0,0.0000,b'2',0.000000,0.056443,0.439155,0.003467,0.422915,0.414912,b'UP'
1,0.0000,b'2',0.021277,0.051699,0.415055,0.003467,0.422915,0.414912,b'UP'
2,0.0000,b'2',0.042553,0.051489,0.385004,0.003467,0.422915,0.414912,b'UP'
3,0.0000,b'2',0.063830,0.045485,0.314639,0.003467,0.422915,0.414912,b'UP'
4,0.0000,b'2',0.085106,0.042482,0.251116,0.003467,0.422915,0.414912,b'DOWN'
...,...,...,...,...,...,...,...,...,...
45307,0.9158,b'7',0.914894,0.044224,0.340672,0.003033,0.255049,0.405263,b'DOWN'
45308,0.9158,b'7',0.936170,0.044884,0.355549,0.003072,0.241326,0.420614,b'DOWN'
45309,0.9158,b'7',0.957447,0.043593,0.340970,0.002983,0.247799,0.362281,b'DOWN'
45310,0.9158,b'7',0.978723,0.066651,0.329366,0.004630,0.345417,0.206579,b'UP'


### Convertendo as colunas categóricas para Bytes de strings


In [25]:
for col in df.select_dtypes([object]):
    df[col] = df[col].str.decode("utf-8")

df

,date,day,period,nswprice,nswdemand,vicprice,vicdemand,transfer,class
0,0.0000,2,0.000000,0.056443,0.439155,0.003467,0.422915,0.414912,UP
1,0.0000,2,0.021277,0.051699,0.415055,0.003467,0.422915,0.414912,UP
2,0.0000,2,0.042553,0.051489,0.385004,0.003467,0.422915,0.414912,UP
3,0.0000,2,0.063830,0.045485,0.314639,0.003467,0.422915,0.414912,UP
4,0.0000,2,0.085106,0.042482,0.251116,0.003467,0.422915,0.414912,DOWN
...,...,...,...,...,...,...,...,...,...
45307,0.9158,7,0.914894,0.044224,0.340672,0.003033,0.255049,0.405263,DOWN
45308,0.9158,7,0.936170,0.044884,0.355549,0.003072,0.241326,0.420614,DOWN
45309,0.9158,7,0.957447,0.043593,0.340970,0.002983,0.247799,0.362281,DOWN
45310,0.9158,7,0.978723,0.066651,0.329366,0.004630,0.345417,0.206579,UP


## 1. Seleção dos Dados


Como vimos na etapa CRISP anterior, alguns dos dados não apresentaram valores reais e foram preenchidos utilizando a média, como nas features **vicdemand**, **vicprice** e **transfer**, cujos dados reais começam apenas após o primeiro ano. Isso pode ser observado nas imagens abaixo, que ilustram a variação de algumas dessas variáveis ao longo do tempo.

![Gráfico](../images/transfer.png)

![Gráfico](../images/vicdemand.png)

![Gráfico](../images/vcprice.png)

Essa característica do conjunto de dados pode impactar diretamente a modelagem preditiva, pois períodos preenchidos com valores artificiais podem introduzir padrões artificiais, afetando a capacidade de generalização dos modelos.

Com isso, optamos em selecionar apenas os dados que houve uma medição real para a próxima fase, a fim de garantir um treinamento com dados mais próximo dos acontecimentos reais.

Para a realizar esse filtro, verificamos até quando o valor de vicprice não variava, assim na primeira vez que ele deixou de possui o valor 0.003467, indicava que ele teve uma medição real. Esse comportamento ocorreu no dia 363, aproximadamente um ano após o início da coleta.

In [26]:
# Encontrar o último valor sem medições
df.loc[df['vicprice'] != 0.003467].iloc[0]

date         0.442326
day                 1
period            0.0
nswprice     0.046325
nswdemand    0.298274
vicprice     0.003232
vicdemand    0.296737
transfer     0.500526
class            DOWN
Name: 17424, dtype: object

In [27]:
# Selecionar apenas os dados com medições reais (após o primeiro ano)
df = df.iloc[17424:]
df

,date,day,period,nswprice,nswdemand,vicprice,vicdemand,transfer,class
17424,0.442326,1,0.000000,0.046325,0.298274,0.003232,0.296737,0.500526,DOWN
17425,0.442326,1,0.021277,0.045485,0.253794,0.003145,0.268255,0.500526,DOWN
17426,0.442326,1,0.042553,0.047316,0.231032,0.002910,0.306577,0.500526,DOWN
17427,0.442326,1,0.063830,0.036658,0.183130,0.002956,0.271621,0.500526,DOWN
17428,0.442326,1,0.085106,0.041882,0.148319,0.002659,0.219834,0.500526,DOWN
...,...,...,...,...,...,...,...,...,...
45307,0.915800,7,0.914894,0.044224,0.340672,0.003033,0.255049,0.405263,DOWN
45308,0.915800,7,0.936170,0.044884,0.355549,0.003072,0.241326,0.420614,DOWN
45309,0.915800,7,0.957447,0.043593,0.340970,0.002983,0.247799,0.362281,DOWN
45310,0.915800,7,0.978723,0.066651,0.329366,0.004630,0.345417,0.206579,UP


A seguir está a distribuição dessas variáveis após a seleção do período com medições reais:

![gráfico](../images/transfer_pos.png)

![Gráfico](../images/vicdemand_pos.png)

![Gráfico](../images/vicprice_pos.png)

Além disso, foi identificado que a normalização prévia dos dados, sem acesso aos valores originais, pode ter influenciado na distribuição das variáveis contínuas, dificultando a identificação de outliers e padrões reais.

Neste momento, optamos por testar os dados com os outliers presentes, dado que a quantidade é pequena e não parece ser um fator crítico. Caso observemos dificuldades nos resultados ou no desempenho dos modelos durante as etapas posteriores, reavaliamos a remoção desses outliers. Também não optamos por retirar nenhuma variável, mas queremos experimentar retirando as variáveis de Day e Date. O foco agora será na análise de como esses dados impactam a performance, sendo possível que ajustes sejam feitos posteriormente, dependendo dos resultados obtidos.

## 2. Limpeza dos Dados 

Assim como informado anteriormente, o dataset disponibilizado já está normalizado, o que dispensa a necessidade de realizar uma nova normalização. 

Além disso, é importante destacar que a remoção de outliers após a normalização pode não ser a abordagem mais recomendada, visto que a normalização se baseia nos valores do dataset, então os outliers que estão sendo removidos já podem ter interferido na distribuição dos dados.

Dessa forma, inicialmente, realizaremos testes com o conjunto de dados completo e, em etapas posteriores, avaliaremos o impacto da remoção de outliers, para verificar se essa prática pode melhorar os resultados para o problema em questão.

Além disso, é importante citar também que o dataset não possui nenhum valor ausente, diante disso não foi necessário utilizar técnicas de remoção ou preenchimento de valores ausentes durante o processo de preparação e limpeza dos dados.

## 3. Construção de Dados

### 3.1 Atributos Derivados

Tendo em vista que o dataset já está completo, para os testes iniciais, não será necessário criar novos atributos derivados. No entanto, considerando que se trata de uma base temporal, uma abordagem interessante a ser explorada em etapas futuras seria a criação de atributos baseados no comportamento da variável alvo nos dias anteriores.  

A tendência de alta ou queda do preço na medição anterior pode ser um fator relevante na previsão do movimento de preço nos dias seguintes, e a inclusão dessa informação pode melhorar a acuracidade do modelo preditivo. Esse é um dos testes que queremos gerar nos modelos, a partir da base de dados gerada nesse relatório.

### 3.2 Registros Gerados

Até o momento, nenhum registro foi gerado, devido à natureza do dataset utilizado. O conjunto de dados original não apresenta a necessidade de registros adicionais durante as etapas de preparação e pré-processamento, uma vez que ele já contém as informações necessárias para a execução dos modelos de machine learning e são informações advindas de coletas do mundo real. 

Além disso, a distribuição dos dados em relação a variável alvo no conjunto de treinamento não é tão desproporcional, por esse motivo na primeira etapa não vamos focar em balancear o dataset.

![DistribuiçãoClasseAlvo](../images/distribuicao_class.png)

No entanto, em etapas futuras pode ser interessante utilizar técnicas de balanceamento no conjunto de treinamento, como um Random Oversampling, a fim de verificar se essa distribuição está impactando nos resultados obtidos (É importante ressaltar que a aplicação de técnicas de balanceamento devem ser feitas apenas no conjunto de treinamento, visto que os conjuntos de validação de de teste devem representar da distribuição real, que não será balanceada).

## 4. Integração de Dados

No momento da escrita desse relatório, não realizaremos nenhuma integração externa com o dataset, e não pretendemos também uma vez que ele o dataset já veio completo. 

## 5. Formatação de Dados

Como discutido nos tópicos anteriores, não houve necessidade de formatação adicional nos dados, pois estes já estavam normalizados quando obtidos da fonte. Dessa forma, essa etapa não precisou ser executada durante o processo de preparação dos dados. 

No entanto, foi necessário realizar uma pequena alteração na estrutura dos dados da classe alvo, uma vez que estava no formato de texto e precisávamos convertê-la para formato numérico, para que ela pudesse ser utilizada nos modelos de machine learning. 

Assim, as variáveis associadas ao valor "UP" receberam o valor numérico 1, enquanto as associadas ao valor "DOWN" foram atribuídas com o valor 0. Gerando o seguinte data-frame.

Antes:

![](../images/class_ant.png)

Depois:

![](../images/class_atu.png)

### 5.1. Separação dos Dados

Nesta fase, o dataset resultante das etapas anteriores foi dividido em três conjuntos: treinamento, validação e teste. O dataset foi particionado da seguinte forma: 65% dos dados foram destinados ao treinamento, 15% à validação e 20% ao teste. Conforme o formato abaixo:

In [28]:
train_df, val_test_df = train_test_split(df, test_size=0.30, random_state=42)
val_df, test_df = train_test_split(val_test_df, test_size=0.50, random_state=42)

In [29]:
# Conjunto de treino
X_train = train_df.drop('class', axis=1) 
Y_train = train_df['class']

# Conjunto de validação
X_val = val_df.drop('class', axis=1) 
Y_val = val_df['class']

# Conjunto de teste
X_test = test_df.drop('class', axis=1) 
Y_test = test_df['class']

In [30]:
# Salvando os conjuntos de dados em arquivos CSV separadamente
X_train.to_csv('../data/processed/X_train.csv', index=False)
Y_train.to_csv('../data/processed/Y_train.csv', index=False)

X_val.to_csv('../data/processed/X_val.csv', index=False)
Y_val.to_csv('../data/processed/Y_val.csv', index=False)

X_test.to_csv('../data/processed/X_test.csv', index=False)
Y_test.to_csv('../data/processed/Y_test.csv', index=False)

Após a divisão, os dados foram salvos em arquivos CSV, que serão utilizados nas etapas subsequentes do processo.